## 1. Import

In [2]:
import pandas as pd
import numpy as np

#StopWords Removal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split
import nltk

import textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Import Data Set

In [3]:
McDonalds_Df_Raw = pd.read_csv("McDonald_s_Reviews.csv", encoding="latin-1")
McDonalds_Df_Raw

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars


## 2. Organize Data

Remove unnecessary information

In [4]:
McDonalds_Df_Raw = McDonalds_Df_Raw.set_index("reviewer_id")
McDonalds_Df_Raw = McDonalds_Df_Raw.drop(columns = ["store_name", "category"])
McDonalds_Df_Raw

,store_address,latitude,longitude,rating_count,review_time,review,rating
reviewer_id,,,,,,,
1,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
2,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
3,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
4,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
5,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...
33392,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33393,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33394,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars


## 3. Keywords Frequency and Ratings

Select columns needed

In [5]:
df_keyword_ratings = McDonalds_Df_Raw.get(["review", "rating"])
df_keyword_ratings

,review,rating
reviewer_id,,
1,Why does it look like someone spit on my food?...,1 star
2,It'd McDonalds. It is what it is as far as the...,4 stars
3,Made a mobile order got to the speaker and che...,1 star
4,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
5,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...
33392,They treated me very badly.,1 star
33393,The service is very good,5 stars
33394,To remove hunger is enough,4 stars


Process dataframe

In [6]:
df_keyword_ratings = df_keyword_ratings.assign(rating = McDonalds_Df_Raw.get("rating").apply(lambda x: int(x[0])))
df_keyword_ratings = df_keyword_ratings.sort_values(by = ["rating"])
df_keyword_ratings

,review,rating
reviewer_id,,
1,Why does it look like someone spit on my food?...,1
13230,The do not offer free refills on your drinks i...,1
24392,"Terrible service, waited forever for our food,...",1
7067,Probably one of the worst experiences I have h...,1
7068,I paid 1 hotcakes and 1 hash brown by drive th...,1
...,...,...
24843,"Everything was cooked perfectly, and the Ser...",5
12364,Love it,5
24845,Im pretty sure that this is still the largest ...,5


4. Removing Stopwords

In [7]:

import nltk

stop_words = stopwords.words('english')
def remove_stop_words(review):
    '''
    Takes in a string comment or review returns a string that has the stopwords removed. For better comprehension of keywords.
    '''
    phrase = review.split()
    stripped_phrase = []
    for word in phrase:
        if word not in stop_words:
            stripped_phrase.append(word)

    return " ".join(stripped_phrase)


In [8]:
def remove_undecoded(review):
    '''
    removes the undecoded texts such as "�" in the original csv file. 
    '''
    return re.sub('[^0-9a-zA-Z\s]+', "",review)
    


In [9]:
#df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_stop_words))
df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_undecoded))
df_nostop_keywords = df_nostop_keywords.drop(columns = "review")

# Exporting the processed file into CSV
df_nostop_keywords.to_csv("review_without_stop_words", encoding = "latin-1")

In [10]:
# df to array put array
# import the training test split. 80 20, 60 40 70 30 <- use package
#
review = np.array(df_nostop_keywords.get("no_stop_words_review"))

In [11]:


def sentimentation(sentences):
    analyzer = SentimentIntensityAnalyzer()
    #compound_score = sentiment['compound']
    # Break down the text into sentences
    sentences = nltk.sent_tokenize(sentences)

    # Analyze sentiment for each sentence
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)

        return sentiment['compound'] 

In [12]:
df_sentiment_nostop_keywords = df_nostop_keywords.sample(n=100)

df = df_sentiment_nostop_keywords.assign(calculated_sent = df_sentiment_nostop_keywords.get("no_stop_words_review").apply(sentimentation))
x = np.array(df.get("no_stop_words_review"))
y = np.array(df.get("rating"))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.50, stratify=y)#change back to 0.20
df['data_type'] = ['not_set']*df.shape[0]
#df.loc[X_train, 'data_type'] = 'train'
# df.loc[X_val, 'data_type'] = 'test'
df
# df.groupby(['Conference', 'label', 'data_type']).count()

,rating,no_stop_words_review,calculated_sent,data_type
reviewer_id,,,,
30603,2,Adopting the kiosks eliminated any chance for ...,0.5859,not_set
11802,3,Its alright,0.2500,not_set
18582,4,Denton,0.0000,not_set
28334,5,Do you feel as bored as my friend is well this...,0.9362,not_set
30410,1,The dining room said open until 330am and it w...,0.0000,not_set
...,...,...,...,...
13750,5,This McDonald,0.0000,not_set
17605,5,Good food,0.4404,not_set
8351,5,Clean and fast,0.4019,not_set


In [13]:
# df_rating_stratfied = df_nostop_keywords.set_index()[df_nostop_keywords.get("rating") == 1]
# df_rating_stratified

In [14]:
# np.random.seed(1)
# df_sampled_1 = df_rating_1.sample(n=10)
# df_sampled_1

In [15]:
# df_sampled_1.assign(sentiment = df_sampled_1.get("no_stop_words_review").apply(sentimentation))

In [16]:
# df_rating_2 = df_nostop_keywords[df_nostop_keywords.get("rating") == 2]
# df_rating_2

In [17]:
# np.random.seed(1)
# df_sampled_2 = df_rating_2.sample(n=10)
# df_sampled_2

In [18]:
# df_sampled_2.assign(sentiment = df_sampled_2.get("no_stop_words_review").apply(sentimentation))